In [6]:
import shutil
import os

import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

# better pictures and legends
plt.rc('figure', dpi=100)
plt.rc('text', usetex=True)
plt.rc('font', family='serif', size=10)

In [3]:
def add_ghosts(y, ghosts):
    shape = np.array(y.shape) + 2*ghosts # Two sides for each axis
    ndim = y.ndim
    yg = np.zeros(shape)
    if ndim == 1:
        yg[ghosts:-ghosts] = y
    if ndim == 2:
        yg[ghosts:-ghosts,ghosts:-ghosts] = y
    if ndim == 3:
        yg[ghosts:-ghosts,ghosts:-ghosts,ghosts:-ghosts] = y
    return yg

def apply_boundary(y, type, ghosts):
    ndim = y.ndim
    if type == "periodic":
        if ndim == 1:
            y[:ghosts] = y[-2*ghosts:-ghosts] # Left
            y[-ghosts:] = y[ghosts:2*ghosts] # Right
        if ndim == 2:
            y[:ghosts,:] = y[-2*ghosts:-ghosts,:]
            y[-ghosts:,:] = y[ghosts:2*ghosts,:]
            y[:,:ghosts] = y[:,-2*ghosts:-ghosts]
            y[:,-ghosts:] = y[:,ghosts:2*ghosts]
    else:
        raise Exception("unknown boundary condition")
    return y

def deghost(y, ghosts):
    if y.ndim == 1:
        return y[ghosts:-ghosts]
    if y.ndim == 2:
        return y[ghosts:-ghosts,ghosts:-ghosts]

# Perform tests for the above functions

a = np.array(
    [
        [1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]
    ]
)

b = add_ghosts(a, ghosts=1)
print(b)

print(apply_boundary(b, "periodic", ghosts=1))
print(deghost(b, ghosts=1))

[[0. 0. 0. 0. 0.]
 [0. 1. 2. 3. 0.]
 [0. 4. 5. 6. 0.]
 [0. 7. 8. 9. 0.]
 [0. 0. 0. 0. 0.]]
[[9. 7. 8. 9. 7.]
 [3. 1. 2. 3. 1.]
 [6. 4. 5. 6. 4.]
 [9. 7. 8. 9. 7.]
 [3. 1. 2. 3. 1.]]
[[1. 2. 3.]
 [4. 5. 6.]
 [7. 8. 9.]]


In [8]:
try:
    shutil.rmtree("frames")
except:
    pass

L = 1
N = 200
ghosts = 1
bc = "periodic"

x, h = np.linspace(0, L, N, retstep=True, endpoint=False)
y, _  = np.linspace(0, L, N, retstep=True, endpoint=False)

xv, yv = np.meshgrid(x, y)
xv = add_ghosts(xv, ghosts)
xv = apply_boundary(xv, "periodic", ghosts)
yv = add_ghosts(yv, ghosts)
yv = apply_boundary(yv, "periodic", ghosts)


def prime(u, axis):
    u_prime = np.zeros_like(u)
    if axis == "x":
        u_prime[ghosts:-ghosts,ghosts:-ghosts] = (u[2:,ghosts:-ghosts]-u[:-2,ghosts:-ghosts]) / (2*h)
    if axis == "y":
        u_prime[ghosts:-ghosts,ghosts:-ghosts] = (u[ghosts:-ghosts,2:]-u[ghosts:-ghosts,:-2]) / (2*h)
    return u_prime

def pprime(u, axis):
    u_prime = np.zeros_like(u)
    if axis == "x":
        u_prime[ghosts:-ghosts,ghosts:-ghosts] = (u[2:,ghosts:-ghosts] - 2*u[1:-1,ghosts:-ghosts] + u[:-2,ghosts:-ghosts]) / h**2
    if axis == "y":
        u_prime[ghosts:-ghosts,ghosts:-ghosts] = (u[ghosts:-ghosts,2:] - 2*u[ghosts:-ghosts,1:-1] + u[ghosts:-ghosts,:-2]) / h**2
    return u_prime

def create_EFE_image():
    fig = plt.figure(figsize=(1,1), dpi=N)
    fig.text(0.17, 0.5, r"$G_{\mu\nu}= 8 \pi T_{\mu\nu}$", fontsize=9, va="top")
    fig.canvas.draw()
    img = np.abs(1-np.asarray(fig.canvas.renderer.buffer_rgba())[:,:,0]/255)
    plt.close(fig)
    return img

## Create a plot of Einstein field equations
u0 = create_EFE_image()[::-1]
print(u0.shape)
if u0.shape != (N, N):
    raise Exception("wrong image size")

## ... or load an image
# from matplotlib.image import imread
# img = imread("Glenda, the Plan 9 Bunny by Renée French.png")
# u0 = np.abs(1-img[::-1])



u0 = add_ghosts(u0, ghosts)
u0 = apply_boundary(u0, bc, ghosts)


U0 = {
    "u": u0,
    "A": np.zeros_like(u0)
}

variables = U0.keys()

U0 = {var: apply_boundary(U0[var], bc, ghosts) for var in variables}


## Evolution
CFL = 0.4
dt = CFL * h

t = 0
t_end = 0.8
i = 0

def rhs(U, key):
    RHS = {
        "u": lambda: U["A"],
        "A": lambda: pprime(U["u"], axis="x") + pprime(U["u"], axis="y"),
    }
    y = RHS[key]()
    y = apply_boundary(y, bc, ghosts)
    return y

U = U0.copy()

while t < t_end:

    ax = plt.gca()
    ax.set_aspect('equal')
    im = ax.pcolormesh(deghost(xv, ghosts), deghost(yv, ghosts), deghost(U["u"], ghosts),
                       vmin=-1.2, vmax=1.2, cmap='viridis')
    plt.colorbar(im)
    plt.title(f't={t:.2f}')
    os.makedirs("frames", exist_ok=True)
    plt.savefig(f"frames/frame.{i:04d}.png")
    plt.close('all')

    print(f"Solving at {t}")
    Unext = {}

    # RK4
    k1 = {var: rhs(U, var) for var in variables}

    k2 = {var: rhs({var: U[var] + dt/2*k1[var] for var in variables}, var) for var in variables}
    
    k3 = {var: rhs({var: U[var] + dt/2*k2[var] for var in variables}, var) for var in variables}

    k4 = {var: rhs({var: U[var] + dt*k3[var] for var in variables}, var ) for var in variables}


    Unext = {var: U[var] + (k1[var]/6 + k2[var]/3 + k3[var]/3 + k4[var]/6)*dt for var in variables}
    for key in Unext.keys():
        Unext[key] = apply_boundary(Unext[key], bc, ghosts)

    U = Unext.copy()
    

    t += dt
    i += 1 


os.system("~/tilde/perspective/create_video.py --frame-rate=15 --frames-dir frames --output movie.mp4")
os.system(f'rm frames/*.png')

Solving at 0
Solving at 0.002
Solving at 0.004
Solving at 0.006
Solving at 0.008
Solving at 0.01
Solving at 0.012
Solving at 0.014
Solving at 0.016
Solving at 0.018000000000000002
Solving at 0.020000000000000004
Solving at 0.022000000000000006
Solving at 0.024000000000000007
Solving at 0.02600000000000001
Solving at 0.02800000000000001
Solving at 0.030000000000000013
Solving at 0.032000000000000015
Solving at 0.034000000000000016
Solving at 0.03600000000000002
Solving at 0.03800000000000002
Solving at 0.04000000000000002
Solving at 0.04200000000000002
Solving at 0.044000000000000025
Solving at 0.04600000000000003
Solving at 0.04800000000000003
Solving at 0.05000000000000003
Solving at 0.05200000000000003
Solving at 0.054000000000000034
Solving at 0.056000000000000036
Solving at 0.05800000000000004
Solving at 0.06000000000000004
Solving at 0.06200000000000004
Solving at 0.06400000000000004
Solving at 0.06600000000000004
Solving at 0.06800000000000005
Solving at 0.07000000000000005
Solvi

ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with clang version 15.0.7
  configuration: --prefix=/Users/runner/miniforge3/conda-bld/ffmpeg_1696213807101/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_pl --cc=arm64-apple-darwin20.0.0-clang --cxx=arm64-apple-darwin20.0.0-clang++ --nm=arm64-apple-darwin20.0.0-nm --ar=arm64-apple-darwin20.0.0-ar --disable-doc --disable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libfontconfig --enable-libopenh264 --enable-libdav1d --enable-cross-compile --arch=arm64 --target-os=darwin --cross-prefix=arm64-apple-darwin20.0.0- --host-cc=/Users/runner/miniforge3/conda-bld/ffmpeg_1696213807101/_build_env/bin/x86_64-apple-darwin13.4.0-clang --enable-neon --enable-gnutls --enable-libmp3lame --enable-libvpx --enable-libass --enable-pthreads --enab

0